# Découpage de table de fait et dimensions et préparer table currency manuellement

## Importation des bibilothèqes 

In [2]:
import pandas as pd 

In [ ]:
# Nom du gros fichier 
input_dataset = "synthetic_fraud_data.csv"

## Traitement des valeurs unknown directement sur le fichier source

In [ ]:
# Dictionnaire pays → capitales
capital_map = {
    "UK": "London",
    "Brazil": "Brasilia",
    "Japan": "Tokyo",
    "Australia": "Canberra",
    "Nigeria": "Abuja",
    "Germany": "Berlin",
    "Mexico": "Mexico City",
    "Russia": "Moscow",
    "France": "Paris",
    "Canada": "Ottawa",
    "Singapore": "Singapore"
}

### Travail sur un petit échantiollons de 100 lignes 

In [ ]:
# Lire uniquement les 100 premières lignes pour tester
small_df = pd.read_csv(r"C:\Users\vakch\Desktop\Formation Data Analyst\Projet\synthetic_fraud_data.csv", nrows=100)

# Vérifier le début du dataset
small_df[["city", "country"]].sample(5)

In [ ]:
for country, capital in capital_map.items():
    mask = (small_df["city"] == "Unknown City") & (small_df["country"] == country)
    small_df.loc[mask, "city"] = capital

In [ ]:
small_df[["city", "country"]].head(30)

### Travail sur le gros fichier initial 

In [ ]:
output_file = "transactions_clean.csv"
# Taille des morceaux : 500 000 lignes 
chunksize = 500_000
first_chunk = True

for chunk in pd.read_csv(input_dataset, chunksize=chunksize):

    # Remplacement automatique via le dictionnaire
    for country, capital in capital_map.items():
        mask = (chunk["city"] == "Unknown City") & (chunk["country"] == country)
        chunk.loc[mask, "city"] = capital

    # Écriture progressive
    if first_chunk:
        chunk.to_csv(output_file, index=False, mode="w")
        first_chunk = False
    else:
        chunk.to_csv(output_file, index=False, mode="a", header=False)

    print(f"✅ Chunk traité : {len(chunk)} lignes")
    
print("✅✅ Fichier final entièrement corrigé.")


In [ ]:
dataset_clean = pd.read_csv(r"C:\Users\vakch\Desktop\Formation Data Analyst\Projet\transactions_clean.csv")
dataset_clean[["city", "country"]].head()

## Découpage de fichier en table de fait et tables de dimensions 

In [3]:
inputfile_balanced = "transactions_rebalanced_pays.csv"
first_chunk = True 
chunk_number = 1

chuck_size = 500_000

for chunk in pd.read_csv(inputfile_balanced,chunksize=chuck_size):
    print(f"✅ Chunk {chunk_number} en cours de traitement...")
    
    #creation des tables de dimensions (Dm)
    df_client = chunk[["customer_id"]]
    df_card = chunk[["card_number","card_type"]]
    df_merchant = chunk[["merchant","merchant_category","merchant_type"]]
    df_localisation = chunk[["city","country","city_size"]]
    df_currency = chunk[["country","currency"]]
    df_channel = chunk[["device","channel"]]
    df_risk_merchant = chunk[["merchant_category","high_risk_merchant"]]
    

    # creation de la table de faits 
    df_transactions =  chunk[[
        "transaction_id", "customer_id","card_number",
        "timestamp","merchant", "amount", "currency",
        "city","card_present","device","device_fingerprint",
        "ip_address", "distance_from_home", "merchant_category",
        "transaction_hour", "weekend_transaction", 
        "velocity_last_hour","is_fraud"
    ]]

    # 1er passage -> ecriture normale (w)
    # ensuite ajouter (a)
    # l'en - tête est écrit une seule fois 

    mode = "w" if first_chunk else "a"
    header = first_chunk

    # ajout progressive des données sans saturer la RAM 
    df_client.to_csv("dm_client_balanced_pays.csv", mode=mode, header=header, index=False)
    df_card.to_csv("dm_card_balanced_pays.csv", mode=mode, header=header, index=False)
    df_merchant.to_csv("dm_merchant_balanced_pays.csv", mode=mode, header=header, index=False)
    df_localisation.to_csv("dm_localisation_balanced_pays.csv", mode=mode, header=header, index=False)
    df_currency.to_csv("dm_currency_balanced_pays.csv", mode=mode, header=header, index=False)
    df_channel.to_csv("dm_channel_balanced_pays.csv", mode=mode, header=header, index=False)
    df_risk_merchant.to_csv("dm_risk_merchant_balanced_pays.csv", mode=mode, header=header, index=False)
    df_transactions.to_csv("fact_transactions_balanced_pays.csv", mode=mode, header=header, index=False)

    first_chunk = False

    chunk_number += 1
print("Découpage terminé avec succés")

✅ Chunk 1 en cours de traitement...
✅ Chunk 2 en cours de traitement...
✅ Chunk 3 en cours de traitement...
✅ Chunk 4 en cours de traitement...
✅ Chunk 5 en cours de traitement...
✅ Chunk 6 en cours de traitement...
✅ Chunk 7 en cours de traitement...
✅ Chunk 8 en cours de traitement...
✅ Chunk 9 en cours de traitement...
✅ Chunk 10 en cours de traitement...
✅ Chunk 11 en cours de traitement...
✅ Chunk 12 en cours de traitement...
✅ Chunk 13 en cours de traitement...
Découpage terminé avec succés


In [ ]:
localisations_clean =  pd.read_csv(r"C:\Users\vakch\Desktop\Formation Data Analyst\Projet\dm_localisation.csv")
localisations_clean

 ### Pour la table currrency, on nettoie ici car on veut récupérer les combinaisons de pays, currency unique pour pouvoir mettre les cofficients de reversion en euros et dollars 

In [ ]:
input_file = "dm_currency.csv"
output_file = "dm_currency_sans_doublons.csv"

df = pd.read_csv(input_file)

# Supprimer les doublons sur (country, currency)
df_unique = df.drop_duplicates(subset=["country","currency"])

# Sauvegarde
df_unique.to_csv(output_file, index=False)

print("✅ Doublons supprimés")
print(f"Lignes finales : {len(df_unique)}")


In [ ]:
df_currency_sans_doublons = pd.read_csv(r"C:\Users\vakch\Desktop\Formation Data Analyst\Projet\dm_currency_sans_doublons.csv")
df_currency_sans_doublons